In [17]:
import os

import pandas as pd
import numpy as np

import findspark
findspark.init()
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window as W
from pyspark.sql import SparkSession

import spark_df_profiling

In [18]:

n_cores =4
SPARK_DATA = os.environ.get('SPARK_DATA')
DRIVER_MEMORY = '10g'  # this is virtual, it's not used until you use it
conf = (pyspark.SparkConf().setAppName('').setMaster(
    'local[{}]'.format(n_cores))
        .set('spark.driver.memory', DRIVER_MEMORY)
#         .set('spark.driver.extraJavaOptions', '-Dderby.system.home={} -Duser.timezone=GMT'.format(SPARK_DATA))
#         .set('spark.executor.extraJavaOptions', '-Duser.timezone=GMT')
#         .set('spark.sql.warehouse.dir', '{}/warehouse'.format(SPARK_DATA))
#         .set('spark.local.dir', '{}/local'.format(SPARK_DATA))
        .set('spark.network.timeout', '300s')
        .set("spark.driver.maxResultSize", "10")
        )

spark = (pyspark.sql.SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate())

spark

In [28]:
df=pd.read_csv("test.csv",)
df = df.drop(columns=['Cabin'])
sdf= spark.createDataFrame(df)
sdf.printSchema()

root
 |-- PassengerId: long (nullable = true)
 |-- Pclass: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: long (nullable = true)
 |-- Parch: long (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)



In [30]:
profile = spark_df_profiling.ProfileReport(sdf, check_correlation=True)

/Applications/miniconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:1238: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))


In [32]:
#profile

In [36]:
sdf_week = spark.createDataFrame(df_week, "id: int, date: date, a: int, b: int")
sdf_week.printSchema()

root
 |-- id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- a: integer (nullable = true)
 |-- b: integer (nullable = true)



## create TS
- the pivot thing work only if you have only 1 colum
- reindex is a join, basically. Let's spark it

In [37]:
df_date_list = pd.DataFrame(pd.date_range(end=pd.Timestamp('2018-07-09'), periods=6, freq='W-MON'), 
                            columns=['date'])
sdf_date_list = spark.createDataFrame(df_date_list, 'date: timestamp')
sdf_date_list.printSchema()
sdf_date_list.toPandas()

root
 |-- date: timestamp (nullable = true)



,date
0,2018-06-04
1,2018-06-11
2,2018-06-18
3,2018-06-25
4,2018-07-02
5,2018-07-09


In [38]:
df_id_list = pd.DataFrame([1, 2], columns=['id'])
sdf_id_list = spark.createDataFrame(df_id_list, 'id: int')
sdf_id_list.printSchema()
sdf_id_list.toPandas()

root
 |-- id: integer (nullable = true)



,id
0,1
1,2


In [39]:
sdf_ts = sdf_id_list.crossJoin(sdf_date_list)
sdf_ts.printSchema()
sdf_ts.toPandas()

root
 |-- id: integer (nullable = true)
 |-- date: timestamp (nullable = true)



,id,date
0,1,2018-06-04
1,1,2018-06-11
2,1,2018-06-18
3,1,2018-06-25
4,1,2018-07-02
5,1,2018-07-09
6,2,2018-06-04
7,2,2018-06-11
8,2,2018-06-18
9,2,2018-06-25


## create ts filling missing dates/id

In [40]:
dates = pd.date_range(end=pd.Timestamp('2018-07-09'), periods=5, freq='W-MON', )
ids = [1, 2]

multi_idx = pd.MultiIndex.from_product([ids, dates, ], names=['id','date'])
multi_idx = multi_idx.delete(2)

df_week = pd.DataFrame({'a': np.random.randint(0, 10, size=len(multi_idx)), 
                   'b': np.random.randint(-5, 5, size=len(multi_idx))
                  }, index=multi_idx)
df_week = df_week.reset_index()
df_week.info(), display(df_week);

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
id      9 non-null int64
date    9 non-null datetime64[ns]
a       9 non-null int64
b       9 non-null int64
dtypes: datetime64[ns](1), int64(3)
memory usage: 368.0 bytes


,id,date,a,b
0,1,2018-06-11,9,2
1,1,2018-06-18,9,0
2,1,2018-07-02,4,2
3,1,2018-07-09,6,-5
4,2,2018-06-11,3,1
5,2,2018-06-18,7,-3
6,2,2018-06-25,3,-5
7,2,2018-07-02,3,-1
8,2,2018-07-09,1,-2


In [41]:
sdf_week = spark.createDataFrame(df_week)
sdf_week.printSchema()

root
 |-- id: long (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- a: long (nullable = true)
 |-- b: long (nullable = true)



### join ts_ref with processed

In [42]:
sdf_week_reindexed = (sdf_ts
                      .join(sdf_week, how='left', on=['id', 'date'])
                      .orderBy(['id', 'date'])
                     )
sdf_week_reindexed.printSchema()
sdf_week_reindexed.toPandas()

root
 |-- id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- a: long (nullable = true)
 |-- b: long (nullable = true)



,id,date,a,b
0,1,2018-06-04,NaN,NaN
1,1,2018-06-11,9.0,2.0
2,1,2018-06-18,9.0,0.0
3,1,2018-06-25,NaN,NaN
4,1,2018-07-02,4.0,2.0
5,1,2018-07-09,6.0,-5.0
6,2,2018-06-04,NaN,NaN
7,2,2018-06-11,3.0,1.0
8,2,2018-06-18,7.0,-3.0
9,2,2018-06-25,3.0,-5.0


In [43]:
sdf_week_reindexed = (sdf_ts
                      .join(sdf_week, how='left', on=['id', 'date'])
                      .orderBy(['id', 'date'])
                      .fillna(0)
                     )
sdf_week_reindexed.printSchema()
sdf_week_reindexed.toPandas()

root
 |-- id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- a: long (nullable = true)
 |-- b: long (nullable = true)



,id,date,a,b
0,1,2018-06-04,0,0
1,1,2018-06-11,9,2
2,1,2018-06-18,9,0
3,1,2018-06-25,0,0
4,1,2018-07-02,4,2
5,1,2018-07-09,6,-5
6,2,2018-06-04,0,0
7,2,2018-06-11,3,1
8,2,2018-06-18,7,-3
9,2,2018-06-25,3,-5


### resample month to week

In [57]:
# # pandas way
# def month_to_week_ts(df, id_level='id', date_level='date_target'):
#     '''
#     - assume is a ts
#     '''
#     # align with week
#     df_month_wmon = df.groupby([pd.Grouper(level=id_level), pd.Grouper(freq='W-MON', level=date_level)]).first()


#     ids = df_month_wmon.index.get_level_values(id_level).unique()
#     dates_start = df_month_wmon.index.get_level_values(date_level).min()
#     dates_end = df_month_wmon.index.get_level_values(date_level).max() + pd.tseries.offsets.MonthBegin(1)  # get begin of the next month
#     dates = pd.date_range(start=dates_start, end=dates_end, freq='W-MON', )

#     # do not fill with zero, but with ffill
#     df_week = make_ts(df_month_wmon, ids=ids, dates=dates, fillna=None, date_name=date_level)
#     df_week = df_week.ffill()

#     return df_week

In [58]:
dates = pd.date_range(end=pd.Timestamp('2018-07-09'), periods=3, freq='MS', )
ids = [1, 2]

multi_idx = pd.MultiIndex.from_product([ids, dates, ], names=['id','date'])
multi_idx = multi_idx.delete(2)

df_month = pd.DataFrame({'a': np.random.randint(0, 10, size=len(multi_idx)), 
                   'b': np.random.randint(-5, 5, size=len(multi_idx))
                  }, index=multi_idx)
df_month = df_month.reset_index()
df_month.info(), display(df_month);

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
id      5 non-null int64
date    5 non-null datetime64[ns]
a       5 non-null int64
b       5 non-null int64
dtypes: datetime64[ns](1), int64(3)
memory usage: 240.0 bytes


,id,date,a,b
0,1,2018-05-01,6,-1
1,1,2018-06-01,9,-2
2,2,2018-05-01,4,-1
3,2,2018-06-01,1,-5
4,2,2018-07-01,7,4


In [59]:
sdf_month = spark.createDataFrame(df_month)
sdf_month.printSchema(), display(sdf_month.toPandas());

root
 |-- id: long (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- a: long (nullable = true)
 |-- b: long (nullable = true)



,id,date,a,b
0,1,2018-05-01,6,-1
1,1,2018-06-01,9,-2
2,2,2018-05-01,4,-1
3,2,2018-06-01,1,-5
4,2,2018-07-01,7,4


##### we need to transform the first on the month in frist monday of the month

In [60]:
# dates_rng = sdf_month.select('date').agg(F.min('date'), F.max('date')).collect()[0]
# date_start_wmon = pd.date_range(start=dates_rng[0], end=dates_rng[0] + pd.DateOffset(days=6), freq='W-MON')
# date_end_wmon = pd.date_range(start=dates_rng[1], end=dates_rng[1] + pd.DateOffset(days=6), freq='W-MON')
# date_start_wmon[0], date_end_wmon

In [61]:
import datetime

In [62]:
sdf_month.withColumn('date_week', F.date_add('date', 1)).toPandas()

,id,date,a,b,date_week
0,1,2018-05-01,6,-1,2018-05-02
1,1,2018-06-01,9,-2,2018-06-02
2,2,2018-05-01,4,-1,2018-05-02
3,2,2018-06-01,1,-5,2018-06-02
4,2,2018-07-01,7,4,2018-07-02


In [63]:
sdf_month.withColumn('date_week', F.when(F.col('date')=='2018-05-01', F.date_add('date', 1))).toPandas()

,id,date,a,b,date_week
0,1,2018-05-01,6,-1,2018-05-02
1,1,2018-06-01,9,-2,None
2,2,2018-05-01,4,-1,2018-05-02
3,2,2018-06-01,1,-5,None
4,2,2018-07-01,7,4,None


In [64]:
d = {'2018-05-01': '2018-05-0',
     '2018-06-01': '2018-06-03',
          '2018-07-01': '2018-06-04',
    }

* doing this way we should have create a F.when() with all the conditions, which is a apin
* cast to string and replae with dictionary?
    - CANNOT USE DATE, so cast to string
    - see func docs

In [65]:
#help(sdf_month2.replace)

In [66]:
sdf_month2 = sdf_month.withColumn('date', F.date_format('date', 'yyyy-MM-dd').cast(StringType()))
dates_months = sdf_month2.select('date').distinct().collect()
# this ugly bit uses date_rage to get the first monday, then  put into dateframe to use cast to string
month2week_d = {r[0]:pd.to_datetime(pd.date_range(start=r[0], end=pd.Timestamp(r[0])+pd.Timedelta(days=6), freq='W-MON')).astype(str)[0] for r in dates_months}
month2week_d
# date_start_wmon = pd.date_range(start=dates_rng[0], end=dates_rng[0] + pd.DateOffset(days=6), freq='W-MON')
# date_end_wmon = pd.date_range(start=dates_rng[1], end=dates_rng[1] + pd.DateOffset(days=6), freq='W-MON')
# date_start_wmon[0], date_end_wmon

{'2018-07-01': '2018-07-02',
 '2018-05-01': '2018-05-07',
 '2018-06-01': '2018-06-04'}

In [67]:
for k, v in month2week_d.items():
    sdf_month2 = sdf_month2.replace(k, v, subset=['date'])
sdf_month2.printSchema(), display(sdf_month2.toPandas());

root
 |-- id: long (nullable = true)
 |-- date: string (nullable = true)
 |-- a: long (nullable = true)
 |-- b: long (nullable = true)



,id,date,a,b
0,1,2018-05-07,6,-1
1,1,2018-06-04,9,-2
2,2,2018-05-07,4,-1
3,2,2018-06-04,1,-5
4,2,2018-07-02,7,4


### we join with ts to make the wekkly, then we need to ffill!
-https://johnpaton.net/posts/forward-fill-spark/

In [68]:
#sdf_ts.toPandas()

In [69]:
sdf_month2_reindexed = (sdf_ts
                      .join(sdf_month2, how='left', on=['id', 'date'])
                      .orderBy(['id', 'date'])
                     )
sdf_month2_reindexed.printSchema()
sdf_month2_reindexed.toPandas()

root
 |-- id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- a: long (nullable = true)
 |-- b: long (nullable = true)



,id,date,a,b
0,1,2018-06-04,9.0,-2.0
1,1,2018-06-11,NaN,NaN
2,1,2018-06-18,NaN,NaN
3,1,2018-06-25,NaN,NaN
4,1,2018-07-02,NaN,NaN
5,1,2018-07-09,NaN,NaN
6,2,2018-06-04,1.0,-5.0
7,2,2018-06-11,NaN,NaN
8,2,2018-06-18,NaN,NaN
9,2,2018-06-25,NaN,NaN


In [71]:
import sys

In [74]:
# define the window
window = W.partitionBy('id').orderBy('date').rowsBetween(-sys.maxsize, 0)

for c in ['a', 'b']:
    sdf_month2_reindexed = sdf_month2_reindexed.withColumn(c+'_ffilled', F.last(sdf_month2_reindexed[c], ignorenulls=True).over(window))


sdf_month2_reindexed.printSchema()
sdf_month2_reindexed.toPandas()

# # define the forward-filled column
# filled_column = 

# # do the fill
# spark_df_filled = spark_df.withColumn('temp_filled_spark', filled_column)


root
 |-- id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- a: long (nullable = true)
 |-- b: long (nullable = true)
 |-- a_ffilled: long (nullable = true)
 |-- b_ffilled: long (nullable = true)



,id,date,a,b,a_ffilled,b_ffilled
0,1,2018-06-04,9.0,-2.0,9,-2
1,1,2018-06-11,NaN,NaN,9,-2
2,1,2018-06-18,NaN,NaN,9,-2
3,1,2018-06-25,NaN,NaN,9,-2
4,1,2018-07-02,NaN,NaN,9,-2
5,1,2018-07-09,NaN,NaN,9,-2
6,2,2018-06-04,1.0,-5.0,1,-5
7,2,2018-06-11,NaN,NaN,1,-5
8,2,2018-06-18,NaN,NaN,1,-5
9,2,2018-06-25,NaN,NaN,1,-5


* well done

## shift date

In [ ]:
w = W.partitionBy('id').orderBy('date')
sdf_week_shifted = sdf_week_reindexed.withColumn('date_target', F.lag('date', count=1).over(w))
sdf_week_shifted.printSchema()
sdf_week_shifted.toPandas()

In [ ]:
w = W.partitionBy('id').orderBy('date')
sdf_week_shifted = (sdf_week_reindexed
                    .withColumn('date_target', F.lag('date', count=1).over(w))
                    .drop('date')
                    .dropna()
                   )
sdf_week_shifted.printSchema()
sdf_week_shifted.toPandas()

## create moving avg

In [ ]:
window_steps = 2
w = W.partitionBy('id').orderBy('date_target').rowsBetween(-window_steps+1, 0) # rows back from current, count from zero
sdf_week_features = sdf_week_shifted.withColumn('a_mov_avg', F.mean('a').over(w))
sdf_week_features.printSchema()
sdf_week_features.toPandas()

In [ ]:
window_steps = 2
w = W.partitionBy('id').orderBy('date_target').rowsBetween(-window_steps+1, 0) # rows back from current, count from zero
sdf_week_features = sdf_week_shifted
for c in ['a', 'b']:
    sdf_week_features = sdf_week_features.withColumn(c+'mov_avg', F.mean(c).over(w))
sdf_week_features.printSchema()
sdf_week_features.toPandas()

#### compare with the pandas 
- we can use the tsfresh

In [ ]:
sdf_week_shifted.toPandas().set_index(['id', 'date_target']).groupby(level='id', group_keys=False).rolling(2).mean()

## create mov diff

In [ ]:
import tsfresh.feature_extraction.feature_calculators as tsf

In [ ]:
help(tsf.mean_change)

In [ ]:
window=3 # this is equivalent to 2 in spark
f = lambda x: np.mean(np.diff(x))
df_feat = sdf_week_shifted.toPandas().set_index(['id', 'date_target']).groupby(level='id', group_keys=False).rolling(window).apply(f, raw=False).add_suffix('_change')
sdf_week_shifted.toPandas().set_index(['id', 'date_target']).join(df_feat)

In [ ]:
window_steps = 2 # this mean that sum of items. In pandas this is 3
w_mean = W.partitionBy('id').orderBy('date_target').rowsBetween(-window_steps+1, 0) # rows back from current, count from zero
#sdf_week_features = sdf_week_shifted.withColumn('a_mov_avg', F.mean('a').over(w))
w_diff = W.partitionBy('id').orderBy('date_target')
sdf_week_features = sdf_week_shifted.withColumn('a_change', F.mean(F.col('a') - F.lag('a', 1).over(w_diff)).over(w_mean))
sdf_week_features.printSchema()
sdf_week_features.toPandas()

In [ ]:
window_steps = 2 # this mean that sum of items. In pandas this is 3
w_mean = W.partitionBy('id').orderBy('date_target').rowsBetween(-window_steps+1, 0) # rows back from current, count from zero
#sdf_week_features = sdf_week_shifted.withColumn('a_mov_avg', F.mean('a').over(w))
w_diff = W.partitionBy('id').orderBy('date_target')
sdf_week_features = sdf_week_shifted.withColumn('a_abs_change', F.mean(F.abs(F.col('a') - F.lag('a', 1).over(w_diff))).over(w_mean))
sdf_week_features.printSchema()
sdf_week_features.toPandas()